### This jupyter note book include scripts
1. export data from redcap 
2. preprocess data into dataframe in format used for drawing Kaplan Meier curve
3. utlize jupytab to provided server contains the tables connect to tableau through web data connector

Reference: Jupytab https://towardsdatascience.com/interactive-simulation-with-tableau-and-jupytab-c26adb1be564

In [5]:
import pandas as pd
import jupytab
import numpy as np
import pandas as pd
import os
import csv
import requests
import sys
from datetime import datetime
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO
import warnings
warnings.filterwarnings("ignore")

#### Export data from redcap 
- we use the script to export records from redcap, the script can get from API Playground in redcap webpage
- when the HTTP STatus as 200, the http request successfully and we store data in CSV format

In [6]:
# this function is used to export data from the redcapdatabase
def export_records_from_redcap(API_token):
    '''
    API_token: the string of the api token
    '''
    data = {
        'token': API_token,
        'content': 'record',
        'action': 'export',
        'format': 'csv',
        'type': 'flat',
        'csvDelimiter': '',
        'rawOrLabel': 'raw',
        'rawOrLabelHeaders': 'raw',
        'exportCheckboxLabel': 'false',
        'exportSurveyFields': 'false',
        'exportDataAccessGroups': 'false',
        'returnFormat': 'json'
    }
    r = requests.post('https://redcap.wehi.edu.au/api/',data=data)
    print('HTTP Status: ' + str(r.status_code))
    data = r.text
    extracted_data = pd.read_csv(StringIO(data), sep=",")
    return extracted_data

In [7]:
# API_token = '5384D3408C5BD6C3611DFFCA1D3B4E7F'
# extracted_data = export_records_from_redcap(API_token)

#### preprocess data to format used for plotting Kaplan Meier curve 

In [8]:
# this function is used to change the format extracted data redcap database format to seperate patient, condition and medication 
def transform_redcapdata_to_synthea(extracted_data):
    '''
    extracted_data: the data extracted from the redcapdatabese in csv format
    '''
    # as the extracted_data from redcap mixed the patient, condition and medication information,
    # we seperate according to the redcap_repeat_instrument column
    Patients = extracted_data[extracted_data['redcap_repeat_instrument'].isnull()]
    Medications = extracted_data[extracted_data['redcap_repeat_instrument'] == 'medications']
    Conditions = extracted_data[extracted_data['redcap_repeat_instrument'] == 'conditions']   
    
    
    # map the column field name from redcap to name as Synthea so that can reuse code in the python scripts used for preprocess and clen data

    Patients = Patients.rename(columns = {'id_patient':'Id',
                                         'deathdate_patient':'DEATHDATE',
                                         'birthdate_patient':'BIRTHDATE',
                                         'gender_patient':'GENDER'})


    Medications = Medications.rename(columns = {'id_patient':'PATIENT',
                                         'description_medication':'DESCRIPTION',
                                               'start_medication':'START',
                                               'reasondescription_medication':'REASONDESCRIPTION',
                                               'code_medication':'CODE',
                                               'reasoncode_medication':'REASONCODE'})


    Conditions = Conditions.rename(columns = {'id_patient':'PATIENT',
                                         'description_condition':'DESCRIPTION',
                                               'code_condition':'CODE',
                                             'start_condition':'START'})
    
    # remove duplicate records
    Patients = Patients.drop_duplicates(keep='first')
    Medications = Medications.drop_duplicates(keep='first')
    Conditions = Conditions.drop_duplicates(keep='first')
    
    return Patients,Medications,Conditions



In [9]:
# #get the medication under the condition
# def get_medication_under_condition(extracted_data,chosen_condition):
#     Patients,Medications,Conditions = transform_redcapdata_to_synthea(extracted_data)
#     selected_condition = Conditions[Conditions['DESCRIPTION'].isin(chosen_condition)]
#     all_data = selected_condition.merge(Medications, how='inner', left_on=['PATIENT', 'DESCRIPTION'], right_on=['PATIENT', 'REASONDESCRIPTION'],suffixes=['_condition','_medication'])
#     all_medicine = np.unique(all_data.DESCRIPTION_medication)
#     return all_medicine



# chosen_condition = ['Malignant neoplasm of breast (disorder)']

# get_medication_under_condition(extracted_data,chosen_condition)  

In [10]:
# the function is used to get the data including (time_to_event, status) can be pluged into kaplanmeier estimator
def get_standard_format_medication(extracted_data,chosen_condition,chosen_medication,survival_analysis_endtime):
    '''
    extracted_data: data extracted from redcapdatabase
    chosen_condition: list of strings that contains the condition chosen for analysis
    chosen_medication: list of strings that contains the medication chosen for analysis
    survival_analysis_endtime: datatime object of the endtime of survival analysis
    
    '''
    Patients,Medications,Conditions = transform_redcapdata_to_synthea(extracted_data)
    
    Medications['DESCRIPTION'] = Medications['DESCRIPTION'].apply(lambda x: x.lower())
    date_format = "%Y-%m-%d"
    
    # condition on the condition(cancer type) and medication
    selected_Conditions = Conditions[Conditions['CODE'].isin(chosen_condition)]

    selected_Medications = Medications[Medications['CODE'].isin(chosen_medication)]

    # join the medication and condition information
    selected_data = selected_Conditions.merge(selected_Medications, how='inner', left_on=['PATIENT', 'CODE'], right_on=['PATIENT', 'REASONCODE'],suffixes=['_condition','_medication'])

    # change to datetime type for time informatio
    selected_data['START_medication'] = selected_data['START_medication'].apply(lambda row : datetime.strptime(str(row)[:10],date_format) if (type(row)==str) else 'not defined' )
    selected_data['START_condition'] = selected_data['START_condition'].apply(lambda row : datetime.strptime(str(row)[:10],date_format) if (type(row)==str) else 'not defined' )
    
    # get the earliest_medication_starttime for the two medicine for each patient
    earliest_medication_starttime = selected_data.groupby(['PATIENT','DESCRIPTION_medication','CODE_medication','DESCRIPTION_condition','START_condition'])['START_medication'].min().reset_index(name='earliest_medication_starttime')
    
    # we only select case that for the patient only take one of the drug
    number_type_drug_took = earliest_medication_starttime.groupby('PATIENT').size().reset_index(name='type_drug_counts')
    # get list of patient took both drug
    patient_took_both_drug = list(number_type_drug_took.loc[number_type_drug_took.type_drug_counts>1].PATIENT)
    # remove patient that took both drug
    earliest_medication_starttime = earliest_medication_starttime[~earliest_medication_starttime['PATIENT'].isin(patient_took_both_drug)]

    
    # join with patient.csv to get deathdate information
    selected_data = Patients.merge(earliest_medication_starttime, how='inner', left_on=['Id'], right_on=['PATIENT'])
    

    #get status for the event(death as 1, alive as 0)
    selected_data['Status'] = selected_data['DEATHDATE'].apply(lambda row : 1 if (type(row)==str) else 0 )
    selected_data['start_time'] = selected_data['earliest_medication_starttime']
    
    # data must have start_time
    selected_data = selected_data[selected_data['start_time'].notna()]
  

    #get end_time for events( if alive then use the survival_analysis_endtime else use the deathtime
    selected_data['end_time'] = selected_data['DEATHDATE'].apply(lambda row : datetime.strptime(str(row)[:10],date_format) if (type(row)==str) else survival_analysis_endtime)
    selected_data['Time'] = (selected_data['end_time'] - selected_data['start_time'])
    selected_data['BIRTHDATE'] = selected_data['BIRTHDATE'].apply(lambda row : datetime.strptime(str(row)[:10],date_format) if (type(row)==str) else 'not defined' )
    selected_data['Age_cancer'] = (selected_data['START_condition'] - selected_data['BIRTHDATE'])
    selected_data['Age_cancer'] = round(selected_data['Age_cancer'].dt.days/365)
    
    selected_data =selected_data.rename(columns = {'DESCRIPTION_medication':'Medication'})
    # get gender as text
    selected_data.GENDER = selected_data.GENDER.map({1: 'M', 2: 'F'})
    
    # Note time is in type of datetime, we need to change to integer
    selected_data['Time'] = selected_data['Time'].dt.days

    
    # Notice we need to add the start point for each group for the Kaplan Meier curve(Survival probability as 1 when Time at 0)
    start_point = False
    if start_point==False:
        for medication_type in np.unique(selected_data.Medication):
            new_row = {"Id":"Added","Time":0,"Status":0,"Medication": medication_type}
            selected_data = selected_data.append(new_row, ignore_index=True)
        start_point = True

    
    return selected_data

### 5 Test benchmark data

In [11]:
# the function is used to get standard format to do Kaplan Meier curve on differentcondition
def get_standard_format_Synthea_condition_case(Patient,Conditions,chosen_condition,Time_format,survival_analysis_endtime):
    
    selected_condition = Conditions[Conditions['DESCRIPTION'].isin(chosen_condition)]

    selected_data = selected_condition.reset_index().merge(Patient, left_on='PATIENT', right_on='Id',
                                          how='left', suffixes=['_condition',''])
 
    selected_data = selected_data[selected_data['START'].notna()]

    # get start time and end time(death date) if alive then end time as undefined
    selected_data['start_time'] = selected_data['START'].apply(lambda row : datetime.strptime(str(row)[:10],Time_format)  )
    selected_data['Status'] = selected_data['DEATHDATE'].apply(lambda row : 1 if (type(row)==str) else 0 )

    selected_data['end_time'] = selected_data['DEATHDATE'].apply(lambda row : datetime.strptime(str(row)[:10],Time_format) if (type(row)==str) else survival_analysis_endtime)
    

    selected_data['Time'] = (selected_data['end_time'] - selected_data['start_time'])
    # Note time is in type of datetime, we need to change to integer
    selected_data['Time'] = selected_data['Time'].dt.days

    
    selected_data['BIRTHDATE'] = selected_data['BIRTHDATE'].apply(lambda row : datetime.strptime(str(row)[:10],Time_format) if (type(row)==str) else 'not defined' )
    selected_data['Age_cancer'] = (selected_data['start_time'] - selected_data['BIRTHDATE'])
    selected_data['Age_cancer'] = round(selected_data['Age_cancer'].dt.days/365)
    
    
    # Notice we need to add the start point for each group for the Kaplan Meier curve(Survival probability as 1 when Time at 0)
    start_point = False
    if start_point==False:
        for condition_type in np.unique(selected_data.DESCRIPTION):
            new_row = {"Id":"Added","Time":0,"Status":0,"DESCRIPTION": condition_type}
            selected_data = selected_data.append(new_row, ignore_index=True)
        start_point = True
    
    return selected_data[['Id','Time','Status','DESCRIPTION','GENDER','Age_cancer']]

    

In [12]:
Patient = pd.read_csv("../../Test Sets/Set A_All die in 1 year/patients.csv")
Conditions = pd.read_csv("../../Test Sets/Set A_All die in 1 year/conditions.csv")

chosen_condition = ['Malignant neoplasm of breast (disorder)']
Time_format = "%d/%m/%Y"
survival_analysis_endtime = datetime.strptime('2023-1-21',"%Y-%m-%d")

set_a_standard_format = get_standard_format_Synthea_condition_case(Patient,Conditions,chosen_condition,Time_format, survival_analysis_endtime)
set_a_standard_format.to_csv(os.path.join('../../Kaplan_Meier_R/preprocessed_data','all_die_1_year.csv'))


In [13]:
Patient = pd.read_csv("../../Test Sets/Set B_ All alive/patients.csv")
Conditions = pd.read_csv("../../Test Sets/Set B_ All alive/conditions.csv")

chosen_condition = ['Malignant neoplasm of breast (disorder)']
Time_format = "%d/%m/%Y"
survival_analysis_endtime = datetime.strptime('2023/1/21',"%Y/%m/%d")

set_b_standard_format = get_standard_format_Synthea_condition_case(Patient,Conditions,chosen_condition,Time_format, survival_analysis_endtime)
set_b_standard_format.to_csv(os.path.join('../../Kaplan_Meier_R/preprocessed_data','all_alive.csv'))

In [14]:
Patient = pd.read_csv("../../Test Sets/Set C_All die in 1 day/patients.csv")
Conditions = pd.read_csv("../../Test Sets/Set C_All die in 1 day/conditions.csv")

chosen_condition = ['Malignant neoplasm of breast (disorder)']
Time_format = "%d/%m/%Y"
survival_analysis_endtime = datetime.strptime('2023/1/21',"%Y/%m/%d")

set_c_standard_format = get_standard_format_Synthea_condition_case(Patient,Conditions,chosen_condition,Time_format, survival_analysis_endtime)
set_c_standard_format.to_csv(os.path.join('../../Kaplan_Meier_R/preprocessed_data','all_die_1_day.csv'))

In [15]:
Patient = pd.read_csv("../../Test Sets/Set D_All 1 year after each other(Staircase)/patients.csv")
Conditions = pd.read_csv("../../Test Sets/Set D_All 1 year after each other(Staircase)/conditions.csv")

chosen_condition = ['Malignant neoplasm of breast (disorder)']
Time_format = "%d/%m/%Y"
survival_analysis_endtime = datetime.strptime('2023/1/21',"%Y/%m/%d")

set_d_standard_format = get_standard_format_Synthea_condition_case(Patient,Conditions,chosen_condition,Time_format, survival_analysis_endtime)
set_d_standard_format.to_csv(os.path.join('../../Kaplan_Meier_R/preprocessed_data','all_die_prop_time.csv'))

In [16]:
Patient = pd.read_csv("../../Test Sets/two_conditions/Patients_g.csv")
Conditions = pd.read_csv("../../Test Sets/two_conditions/conditions_g.csv")

chosen_condition = ["Malignant neoplasm of breast (disorder)","Metastasis from malignant tumor of breast (disorder)"]
Time_format = "%d/%m/%Y"
survival_analysis_endtime = datetime.strptime('2023/1/21',"%Y/%m/%d")

set_g_standard_format = get_standard_format_Synthea_condition_case(Patient,Conditions,chosen_condition,Time_format, survival_analysis_endtime)
set_g_standard_format.to_csv(os.path.join('../../Kaplan_Meier_R/preprocessed_data','two_condition.csv'))

In [17]:
Patient = pd.read_csv("../../Test Sets/Set E_50% die in 5 years/Patients.csv")
Conditions = pd.read_csv("../../Test Sets/Set E_50% die in 5 years/conditions.csv")

chosen_condition = ["Malignant neoplasm of breast (disorder)","Metastasis from malignant tumor of breast (disorder)"]
Time_format = "%d/%m/%Y"
survival_analysis_endtime = datetime.strptime('2023/1/21',"%Y/%m/%d")

set_e_standard_format = get_standard_format_Synthea_condition_case(Patient,Conditions,chosen_condition,Time_format, survival_analysis_endtime)
set_e_standard_format.to_csv(os.path.join('../../Kaplan_Meier_R/preprocessed_data','half_die.csv'))

-------

#### Utilize Jupytab to provide server 
- load dataframe into server 
- render table schema
- get data when run the config.init file

Use command : <code>jupytab --config= path-to-config.in</code> to run the server under the server environment

In [18]:
def get_standard_format(API_token,chosen_medication,chosen_condition,survival_analysis_endtime):
    
    extracted_data = export_records_from_redcap(API_token)
    selected_data = get_standard_format_medication(extracted_data,chosen_condition,chosen_medication,survival_analysis_endtime)
    
    selected_data.to_csv(os.path.join('../../Kaplan_Meier_R/preprocessed_data','two_drug.csv'))
    
    return selected_data #selected_data[['Id','Time','Status','Medication','Age_cancer','GENDER']]



API_token = '5384D3408C5BD6C3611DFFCA1D3B4E7F'
# chosen_medication = ['100 ML Epirubicin Hydrochloride 2 MG/ML Injection':'1732186','Paclitaxel 100 MG Injection':'583214']
chosen_medication = [1732186,583214]
# chosen_condition = ['Malignant neoplasm of breast (disorder)':254837009]
chosen_condition = [254837009]
survival_analysis_endtime = datetime.strptime('2023-1-21', "%Y-%m-%d")
standard_format = get_standard_format(API_token,chosen_medication,chosen_condition,survival_analysis_endtime)


# DATA = get_standard_format(API_token,chosen_condition,chosen_medication,survival_analysis_endtime)
#Load dataframe with Tables method in Jupytab so that it can be exposed to Tableau
tables = jupytab.Tables()
tables['extracted_data'] = jupytab.DataFrameTable("extracted_data", standard_format, include_index=False)

tables['test_set_a'] = jupytab.DataFrameTable("test_set_a", set_a_standard_format, include_index=False)
tables['test_set_b'] = jupytab.DataFrameTable("test_set_b", set_b_standard_format, include_index=False)
tables['test_set_c'] = jupytab.DataFrameTable("test_set_c", set_c_standard_format, include_index=False)
tables['test_set_d'] = jupytab.DataFrameTable("test_set_d", set_d_standard_format, include_index=False)
tables['test_set_g'] = jupytab.DataFrameTable("test_set_g", set_g_standard_format, include_index=False)
tables['test_set_e'] = jupytab.DataFrameTable("test_set_e", set_e_standard_format, include_index=False)



HTTP Status: 200


In [19]:
# GET /schema
tables.render_schema()

[{"id": "extracted_data", "alias": "extracted_data", "columns": [{"id": "Id", "dataType": "string"}, {"id": "redcap_repeat_instrument", "dataType": "string"}, {"id": "redcap_repeat_instance", "dataType": "float"}, {"id": "BIRTHDATE", "dataType": "datetime"}, {"id": "DEATHDATE", "dataType": "string"}, {"id": "ssn_patient", "dataType": "string"}, {"id": "drivers_patient", "dataType": "string"}, {"id": "passport_patient", "dataType": "string"}, {"id": "prefix_patient", "dataType": "string"}, {"id": "first_patient", "dataType": "string"}, {"id": "last_patient", "dataType": "string"}, {"id": "suffix_patient", "dataType": "string"}, {"id": "maiden_patient", "dataType": "string"}, {"id": "marital_patient", "dataType": "float"}, {"id": "race_patient", "dataType": "string"}, {"id": "ethnicity_patient", "dataType": "string"}, {"id": "GENDER", "dataType": "string"}, {"id": "birthplace_patient", "dataType": "string"}, {"id": "address_patient", "dataType": "string"}, {"id": "city_patient", "dataTyp

In [20]:
# GET /data
try:
    tables.render_data(REQUEST)
except NameError:
    print("Not available outside jupytab context")

Not available outside jupytab context
